<a href="https://colab.research.google.com/github/yuvibirdi/ama/blob/main/animalSoundClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /content/drive/MyDrive/animalSoundClassifier_Project
!git clone https://github.com/YashNita/Animal-Sound-Dataset /content/drive/MyDrive/animalSoundClassifier_Project/dataset


README.md
Cloning into '/content/drive/MyDrive/animalSoundClassifier_Project/dataset'...
remote: Enumerating objects: 887, done.
remote: Total 887 (delta 0), reused 0 (delta 0), pack-reused 887
Receiving objects: 100% (887/887), 100.68 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Updating files: 100% (876/876), done.


In [ ]:
!python --version

Python 3.10.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import IPython.display as ipd

audio_path = "/content/drive/MyDrive/animalSoundClassifier_Project/dataset/dog/dog_97.wav"

ipd.Audio(audio_path)

In [ ]:
# import required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder


In [ ]:
# set up path


DATASET_PATH = "/content/drive/MyDrive/animalSoundClassifier_Project/dataset"

labels = [i for i in os.listdir(DATASET_PATH) if not "." in i]

dataDict = {}
for label in labels:
  dataDict[label] = []
  labelFolderName = DATASET_PATH + "/" + label
  for fileName in os.listdir(labelFolderName):
    dataDict[label].append(fileName)

print(dataDict)

{'lion': ['lion_1.wav', 'lion_10.wav', 'lion_11.wav', 'lion_12.wav', 'lion_13.wav', 'lion_14.wav', 'lion_15.wav', 'lion_16.wav', 'lion_17.wav', 'lion_18.wav', 'lion_19.wav', 'lion_2.wav', 'lion_20.wav', 'lion_21.wav', 'lion_22.wav', 'lion_23.wav', 'lion_24.wav', 'lion_25.wav', 'lion_26.wav', 'lion_27.wav', 'lion_28.wav', 'lion_29.wav', 'lion_3.wav', 'lion_30.wav', 'lion_31.wav', 'lion_32.wav', 'lion_33.wav', 'lion_34.wav', 'lion_35.wav', 'lion_36.wav', 'lion_37.wav', 'lion_38.wav', 'lion_39.wav', 'lion_4.wav', 'lion_40.wav', 'lion_41.wav', 'lion_42.wav', 'lion_43.wav', 'lion_44.wav', 'lion_45.wav', 'lion_5.wav', 'lion_6.wav', 'lion_7.wav', 'lion_8.wav', 'lion_9.wav'], 'donkey': ['donkey_1.wav', 'donkey_10.wav', 'donkey_11.wav', 'donkey_12.wav', 'donkey_13.wav', 'donkey_14.wav', 'donkey_15.wav', 'donkey_16.wav', 'donkey_17.wav', 'donkey_18.wav', 'donkey_19.wav', 'donkey_2.wav', 'donkey_20.wav', 'donkey_21.wav', 'donkey_22.wav', 'donkey_23.wav', 'donkey_24.wav', 'donkey_25.wav', 'donkey_

In [ ]:
# # RENAME THE AUDIO FILES TO ENGLISH

# for label in labels:
#   labelFolderName = DATASET_PATH + "/" + label
#   # print(labelFolderName)
#   for fileName in os.listdir(labelFolderName):
#     # print(fileName)
#     x = fileName.split("_")
#     x[0] = label
#     # print(x)
#     newName = labelFolderName + "/" + "_".join(x)
#     # print(newName)
#     oldFileName = labelFolderName + "/" + fileName
#     os.rename(oldFileName, newName)
# # print(dataDict)

In [ ]:
# create pandas dataframe

dataDictModified = {"filename":[], "className":[]}

for i in dataDict.values():
  for j in i:
    dirName = j.split("_")[0]
    dataDictModified["filename"].append(DATASET_PATH + "/" + dirName + "/" + j)
    dataDictModified["className"].append(dirName)

dataset = pd.DataFrame(dataDictModified)
# dataset

In [ ]:
# map class names to class ids

# print(dataset.className.unique())

encoder = LabelEncoder()
encoder.fit(dataset["className"])

dataset["classID"] = encoder.transform(dataset["className"])
# dataset.classID.unique()

In [ ]:
# Preparing audio transformation functions


import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import random


In [ ]:
# # testing something

# testAud = "/content/drive/MyDrive/animalSoundClassifier_Project/dataset/chicken/chicken_1.wav"
# t, sr = torchaudio.load(testAud)
# # print(t)
# t = torch.cat([t,t])
# print(t[:1, :])

In [ ]:
dataset

,filename,className,classID
0,/content/drive/MyDrive/animalSoundClassifier_P...,lion,7
1,/content/drive/MyDrive/animalSoundClassifier_P...,lion,7
2,/content/drive/MyDrive/animalSoundClassifier_P...,lion,7
3,/content/drive/MyDrive/animalSoundClassifier_P...,lion,7
4,/content/drive/MyDrive/animalSoundClassifier_P...,lion,7
...,...,...,...
870,/content/drive/MyDrive/animalSoundClassifier_P...,chicken,2
871,/content/drive/MyDrive/animalSoundClassifier_P...,chicken,2
872,/content/drive/MyDrive/animalSoundClassifier_P...,chicken,2
873,/content/drive/MyDrive/animalSoundClassifier_P...,chicken,2


In [ ]:
class audioUtil():

  @staticmethod
  def importAudio(file):
    signalTensor, sampleRate = torchaudio.load(file)
    return (signalTensor, sampleRate)

  @staticmethod
  def stdChannelize(importedAud, uniChannel):
    signalTensor, sampleRate = importedAud

    if (signalTensor.shape[0] == uniChannel):
      return importedAud

    newSigTensor = signalTensor[:1, :] if (uniChannel == 1) \
     else torch.cat([signalTensor, signalTensor])

    return (newSigTensor, sampleRate)

  @staticmethod
  def stdResample(importedAud, uniSr):
    signalTensor, sampleRate = importedAud

    if (sampleRate == uniSr):
      return importedAud

    signalTensor = torchaudio.transforms.Resample(sampleRate, uniSr)(signalTensor[:1, :])
    if signalTensor.shape[0] > 1:
      signalTensor2 = torchaudio.transforms.Resample(sampleRate, uniSr)(signalTensor[1:, :])
      signalTensor = torch.cat([signalTensor, signalTensor2])

    return (signalTensor, uniSr)


  @staticmethod
  def stdLength(importedAud, uniLen):
    signalTensor, sampleRate = importedAud
    # print(signalTensor)
    numChannels, sigLen = signalTensor.shape
    maxLen = sampleRate//1000 * uniLen

    if sigLen > maxLen:
      signalTensor = signalTensor[:, :maxLen]
    elif sigLen < maxLen:
      Lmargin = random.randint(0, maxLen-sigLen)
      Rmargin = maxLen-sigLen-Lmargin

      LmarginTensor = torch.zeros((numChannels, Lmargin))
      RmarginTensor = torch.zeros((numChannels, Rmargin))

      signalTensor = torch.cat([LmarginTensor, signalTensor, RmarginTensor], 1)
      # print(signalTensor)

    return (signalTensor, sampleRate)

  @staticmethod
  def timeShift(importedAud, shiftThreshold):
    signalTensor, sampleRate = importedAud
    signalLength = signalTensor.shape[1]
    shiftAmount = int(random.random() * shiftThreshold * signalLength)
    return (signalTensor.roll(shiftAmount), sampleRate)

  @staticmethod
  def genSpec(aud, n_mels=64, n_fft=1024, hop_len=None):
      sig,sr = aud
      top_db = 80

      # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
      spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

      # Convert to decibels
      spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
      return (spec)

  @staticmethod
  def augmentSpec(spec, maxMaskPCT=0.1, nFreqMasks=1, nTimeMasks=1):
    _, nMels, nSteps = spec.shape
    mask_value = spec.mean()
    augSpec = spec

    fqMaskParam = maxMaskPCT * nMels
    for _ in range(nFreqMasks):
      augSpec = transforms.FrequencyMasking(fqMaskParam)(augSpec, mask_value)

    timeMaskParam = maxMaskPCT * nSteps
    for _ in range(nTimeMasks):
      augSpec = transforms.TimeMasking(timeMaskParam)(augSpec, mask_value)

    return augSpec

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4

  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)

  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.df.loc[idx, 'filename']
    # Get the Class ID
    class_id = self.df.loc[idx, 'classID']
    aud = audioUtil.importAudio(audio_file)
    # Some sounds have a higher sample rate, or fewer channels compared to the
    # majority. So make all sounds have the same number of channels and same
    # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # result in arrays of different lengths, even though the sound duration is
    # the same.
    reaud = audioUtil.stdResample(aud, self.sr)
    rechan = audioUtil.stdChannelize(reaud, self.channel)
    # print("rechan_output:", rechan)
    dur_aud = audioUtil.stdLength(rechan, self.duration)

    # print("dur_aud output:", dur_aud)

    shift_aud = audioUtil.timeShift(dur_aud, self.shift_pct)
    sgram = audioUtil.genSpec(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = audioUtil.augmentSpec(sgram, maxMaskPCT=0.1, nFreqMasks=2, nTimeMasks=2)
    # print("passed a file! \n")
    return aug_sgram, class_id


In [ ]:

from torch.utils.data import random_split

myds = SoundDS(dataset, DATASET_PATH)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [ ]:

import torch.nn.functional
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Fourth Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)

    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cuda', index=0)

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        #if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))

    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')

num_epochs=50   # Just for demo, adjust this higher.
# training(myModel, train_dl, num_epochs)


In [ ]:
torch.save(myModel, "/content/drive/MyDrive/animalSoundClassifier_Project/saves/model1.pt")

In [ ]:
import torch
myModel=torch.load("/content/drive/MyDrive/animalSoundClassifier_Project/saves/model1.pt")

# ----------------------------
# Inference
# ----------------------------
def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:

      # Get the input features and target labels, and put them on the GPU
      inputs, labels = data[0].to(device), data[1].to(device)

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      # Get predictions
      outputs = model(inputs)
      print(outputs)
      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      print("prediction",prediction)
      print("labels",labels)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]

  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
# inference(myModel, val_dl)

In [ ]:
import IPython.display as ipd

audio_path = "/content/drive/MyDrive/animalSoundClassifier_Project/dataset/lion/lion_2.wav"

ipd.Audio(audio_path)

In [ ]:
def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU
      inputs, labels = data[0].to(device), data[1].to(device)

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      # Get predictions
      outputs = model(inputs)

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      # print(prediction)
      # print(labels)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]

  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

Accuracy: 0.06, Total items: 175


In [ ]:
import torch
myModel=torch.load("/content/drive/MyDrive/animalSoundClassifier_Project/saves/model1.pt")


audPath = "/content/drive/MyDrive/animalSoundClassifier_Project/dataset/chicken/chicken_1.wav"

inputDS = SoundDS(dataset, "/content/drive/MyDrive/animalSoundClassifier_Project/dataset/lion/")
inputData = torch.utils.data.DataLoader(inputDS, batch_size=1, shuffle=False)
c = 0
# print(aug_sgram)
for datapoint in inputData:

  inputs, labels = datapoint[0].to(device), datapoint[1].to(device)
  inputs_m, inputs_s = inputs.mean(), inputs.std()
  inputs = (inputs - inputs_m) / inputs_s
  # print(inputs)
# Get predictions
  outputs = myModel(inputs)

# Get the predicted class with the highest score
  _, prediction = torch.max(outputs,1)
  print(prediction)
  break
  # c += 1 if prediction == 2 else 0
  # break
# print(c/30)

tensor([1], device='cuda:0')
